### Librería

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
import numpy as np
from random import randint
import random
import math
import seaborn as sns
import matplotlib.pyplot as plt

Mounted at /content/drive


### Base de datos

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning/heart_2020_cleaned.csv')

In [ ]:
dataset

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,Yes,27.41,Yes,No,No,7.0,0.0,Yes,Male,60-64,Hispanic,Yes,No,Fair,6.0,Yes,No,No
319791,No,29.84,Yes,No,No,0.0,0.0,No,Male,35-39,Hispanic,No,Yes,Very good,5.0,Yes,No,No
319792,No,24.24,No,No,No,0.0,0.0,No,Female,45-49,Hispanic,No,Yes,Good,6.0,No,No,No
319793,No,32.81,No,No,No,0.0,0.0,No,Female,25-29,Hispanic,No,No,Good,12.0,No,No,No


### Funciones

In [ ]:
def EntropiaTotal(dataset,col_decision):
  contador_Si = contador_No = 0
  for index,rows in dataset.iterrows():
    if rows[col_decision] == 'Yes':
      contador_Si += 1
    elif rows[col_decision] == 'No':
      contador_No += 1
  Total = contador_Si + contador_No
  print('Total:',Total,'---- Contador Yes:',contador_Si, ' ---- Contador No:', contador_No)
  Entropia_Total =-((contador_Si/Total)*(math.log2(contador_Si/Total))) - ((contador_No/Total)*(math.log2(contador_No/Total)))
  return Entropia_Total

In [ ]:
col_decision = 'HeartDisease'
Entropia_Total = EntropiaTotal(dataset,col_decision)
print('Entropia Total:',Entropia_Total)

Total: 319795 ---- Contador Yes: 27373  ---- Contador No: 292422
Entropia Total: 0.4215945911957518


In [ ]:
def formacion_conjunto_datos(dataset,col_decsion_eliminar):
  dataset1 = dataset.drop([col_decsion_eliminar], axis=1)  
  return dataset1

In [ ]:
dataset1 = formacion_conjunto_datos(dataset,col_decision)

In [ ]:
def entropia(contador_No,contador_Si,Total):
    if contador_No == 0 and Total != 0:
      Entropia = -(contador_Si/Total)*(np.log2(contador_Si/Total))
    if contador_Si == 0 and Total != 0:
      Entropia = - (contador_No/Total)*(np.log2(contador_No/Total))
    if contador_No == 0 and contador_Si == 0 and Total == 0:
      Entropia = 0
    if (contador_Si !=0) and (contador_No !=0):  
      Entropia = -(contador_Si/Total)*(np.log2(contador_Si/Total)) - (contador_No/Total)*(np.log2(contador_No/Total))
    return Entropia

In [ ]:
def ganancia(Entropia_Total,P_E):
  Suma = 0
  for i in P_E:
    Suma += -1*i
  G = Entropia_Total + Suma
  return G

In [ ]:
def Nodo(col_valor, columna):
    max_value = None
    max_idx = None
    L = []
    for idx, num in enumerate(col_valor):
        L.append(round(num,4))
    #print(L) 
    if L.count(L[0]) == len(L): #si todos los valores son iguales
      nodo = 'Todos los nodos intermedios'
      max_value = L[0]
      print( 'Nodo:',nodo,'-- Valor Máximo:', round(max_value,4))
    else:
      for idx, num in enumerate(col_valor):
        if (max_value is None or num > max_value):
            max_value = num
            max_idx = idx
      for idx, row in enumerate(columna):
        if idx == max_idx:
          nodo=row
      print( 'Nodo:',nodo,'-- Valor Máximo:', max_value)
    return nodo, max_value

In [ ]:
def AD(dataset,dataset1,col_decision,Entropia_Total):
  df = pd.DataFrame(columns=['Atributo','Observación', '# Yes', '# No', 'Total','Entropia']) #Creación dataframe
  df3 = pd.DataFrame(columns=['Decisión','Atributo','Ganancia']) #Creación dataframe
  df6 = pd.DataFrame(columns=['Transición 1','Nodo Intermedio','Transición 2', 'Nodo Final']) #Creación dataframe
  for n, i in enumerate(dataset1):
    print('Columna: ',i)
    Lists =[]
    P_E = []
    Total = len(dataset1[i].values)
    for List in dataset1[i].values: #Recorrer los valores de la columna en cuestión
      if List not in Lists: #Si el valor no se encuentra en la lista
        Lists.append(List) #Agregar a la lista ese valor
    print('Observaciones de los Atributos:',Lists)
    print('')
    for index,row in enumerate(Lists):
      df1 = dataset[(dataset[i] == row) & (dataset[col_decision] == 'Yes')] #Seleción de ciertas Instancias del dataset
      df2 = dataset[(dataset[i] == row) & (dataset[col_decision] == 'No')] #Seleción de ciertas Instancias del dataset
      contador = contador_Si = contador_No = 0
      for List in df1[i].values: #Recorrer los valores de la columna en cuestión
        if row == List: #Si valor de la instancia es igual al valor de la columna
          contador_Si = contador_Si + 1
      for List in df2[i].values: #Recorrer los valores de la columna en cuestión
        if row == List: #Si valor de la instancia es igual al valor de la columna
          contador_No = contador_No + 1
      T = contador_Si + contador_No #Total 
      P = T/Total #Probabiidad
      #print('Observación:',row,'---', 'Si:', contador_Si, '-- No:', contador_No, '-- Total:', T)
      Entropia = entropia(contador_No,contador_Si,T)
      #print('Probabilidad:',P,'-- Entropia:', Entropia,'-- P_E:',P*Entropia)
      P_E.append(P*Entropia) #Añadir a la lista
      #print('')
      df = df.append({'Atributo': i, 'Observación':row, '# Yes':contador_Si,'# No': contador_No, 'Total': Total,'Entropia':Entropia}, ignore_index=True)
    #print(P_E)
    G = ganancia(Entropia_Total,P_E) 
    #print('Ganacia:', G)
    df3 = df3.append({'Decisión':col_decision,'Atributo':i, 'Ganancia': G}, ignore_index=True)  
  print(df) #Visualizacion de la entropia 
  print('')
  print(df3) #Visualización de la ganancia
  print('')
  nodo_raiz, max_value = Nodo(df3['Ganancia'], df3['Atributo'])
  dataset2 = dataset1.drop([nodo_raiz], axis=1)  

  List_nodo_raiz =[]
  Total = len(dataset1[nodo_raiz].values)
  for List in dataset1[nodo_raiz].values: #Recorrer los valores de la columna en cuestión
    if List not in List_nodo_raiz: #Si el valor no se encuentra en la lista
      List_nodo_raiz.append(List) #Agregar a la lista ese valor
  print('Observaciones de la columna '+nodo_raiz+':',List_nodo_raiz)
  print('')
  for index,row in enumerate(List_nodo_raiz):
    print('Transición 1: ',row)
    #print('')
    df4 = pd.DataFrame(columns=['Atributo','Observación', '# Yes', '# No', 'Total','Entropia']) #Creación dataframe
    df5 = pd.DataFrame(columns=['Decisión','Columna','Ganancia']) #Creación dataframe
    for n, i in enumerate(dataset2):
        #print('Columna: ',i)
        Lists =[]
        P_E = []
        Total = len(dataset1[i].values)
        for List in dataset1[i].values: #Recorrer los valores de la columna en cuestión
          if List not in Lists: #Si el valor no se encuentra en la lista
            Lists.append(List) #Agregar a la lista ese valor
        #print('Observaciones de los Atributos:',Lists)
        #print('')
        for index1,row1 in enumerate(Lists):
          #print(row1) #impresion de todos los atributos
          df1 = dataset[(dataset[nodo_raiz] == row) & (dataset[i] == row1) & (dataset[col_decision] == 'Yes')] #Seleción de ciertas Instancias del dataset
          #print(df1)
          #print('')
          df2 = dataset[(dataset[nodo_raiz] == row) & (dataset[i] == row1) & (dataset[col_decision] == 'No')] #Seleción de ciertas Instancias del dataset
          #print(df2)
          #print('')
          contador = contador_Si = contador_No = 0
          for List in df1[i].values: #Recorrer los valores de la columna en cuestión
            if row1 == List: #Si valor de la instancia es igual al valor de la columna
              contador_Si = contador_Si + 1
          for List in df2[i].values: #Recorrer los valores de la columna en cuestión
            if row1 == List: #Si valor de la instancia es igual al valor de la columna
              contador_No = contador_No + 1
          T = contador_Si + contador_No #Total 
          P = T/Total #Probabiidad
          #print('Atributo:',row ,'-- Observación:',row1,'--', 'Si:', contador_Si, '-- No:', contador_No, '-- Total:', T) 
          Entropia = entropia(contador_No,contador_Si,T)
          #print('Probabilidad:',P,'-- Entropia:', Entropia,'-- P_E:',P*Entropia) 
          P_E.append(P*Entropia) #Añadir a la lista
          df4 = df4.append({'Atributo': row, 'Observación':row1, '# Yes':contador_Si,'# No': contador_No, 'Total': T,'Entropia':Entropia}, ignore_index=True)
        #print(P_E)
        G = ganancia(Entropia_Total,P_E) 
        #print('')
        #print('Ganacia:', G) 
        df5 = df5.append({'Decisión':nodo_raiz,'Columna':i, 'Ganancia': G}, ignore_index=True)
    #print(df4)
    #print('')
    #print(df5)
    #print('')
    nodo, max_value = Nodo(df5['Ganancia'], df5['Columna'])
    print('')

    O_df = [] #Lista para elementos de Transicion 2
    if nodo == 'Todos los nodos intermedios':
      NODO = [] #Lista para nodos intermedios
      for List in df['Atributo'].values: #Recorrer los valores de la columna en cuestión
        if List not in NODO: #Si el valor no se encuentra en la lista
          NODO.append(List) #Agregar a la lista ese valor    
      NODO.remove(nodo_raiz)
      for b in NODO:
        for n, i in enumerate(df['Atributo']): #Recorrer los valores de la columna en cuestión
          if i == b:
            for m,j in enumerate(df['Observación']):
              if m == n:
                O_df.append(j) 
                 
      #Visulización de la tabla
      for k in O_df:  
          for w,y in enumerate(df['Observación']):
            if k == y:
              for p,q in enumerate(df['Atributo']):
                if w ==p:
                  nodo = q
          for n, i in enumerate(df4['Observación']): #Recorrer los valores de la columna en cuestión
            if i == k:
              for m,j in enumerate(df4['# Yes']):
                if m == n:
                  Si = int(j)
              for m,j in enumerate(df4['# No']):
                if m == n:
                  No =int(j)
          if Si> No:
              df6 = df6.append({'Transición 1':row,'Nodo Intermedio': nodo, 'Transición 2':k, 'Nodo Final':'Yes'}, ignore_index=True)  
          if No>Si:
              df6 = df6.append({'Transición 1':row,'Nodo Intermedio': nodo, 'Transición 2':k, 'Nodo Final':'No'}, ignore_index=True)  


    else:
      for n, i in enumerate(df['Atributo']): #Recorrer los valores de la columna en cuestión
        if i == nodo:
          for m,j in enumerate(df['Observación']):
            if m == n:
              O_df.append(j)

      for k in O_df:
        for n, i in enumerate(df4['Observación']): #Recorrer los valores de la columna en cuestión
          if i == k:
            for m,j in enumerate(df4['# Yes']):
              if m == n:
                Si = int(j)
            for m,j in enumerate(df4['# No']):
              if m == n:
                No =int(j)
        if Si> No:
            df6 = df6.append({'Transición 1':row,'Nodo Intermedio': nodo, 'Transición 2':k, 'Nodo Final':'Yes'}, ignore_index=True)  
        if No>Si:
            df6 = df6.append({'Transición 1':row,'Nodo Intermedio': nodo, 'Transición 2':k, 'Nodo Final':'No'}, ignore_index=True)  
  print(df6)  
  return df6  

In [ ]:
dataset_resultado = AD(dataset,dataset1,col_decision,Entropia_Total)

Columna:  BMI
Observaciones de los Atributos: [16.6, 20.34, 26.58, 24.21, 23.71, 28.87, 21.63, 31.64, 26.45, 40.69, 34.3, 28.71, 28.37, 28.15, 29.29, 29.18, 26.26, 22.59, 29.86, 18.13, 21.16, 28.9, 26.17, 25.82, 25.75, 34.34, 31.66, 24.89, 36.58, 25.84, 30.67, 45.35, 19.02, 38.97, 32.98, 24.62, 20.43, 27.37, 21.93, 22.6, 28.13, 25.06, 30.23, 30.34, 20.37, 33.23, 25.11, 20.12, 27.46, 26.61, 31.09, 27.12, 32.61, 32.1, 24.02, 32.55, 46.52, 29.55, 27.2, 28.94, 21.03, 22.31, 31.46, 26.63, 34.01, 22.04, 23.87, 40.0, 21.97, 27.76, 33.75, 25.54, 27.63, 34.33, 28.29, 30.56, 21.14, 29.53, 24.37, 28.25, 23.78, 34.75, 22.67, 32.81, 27.05, 44.29, 36.02, 21.8, 22.05, 23.96, 19.49, 28.35, 19.47, 24.8, 24.68, 23.49, 27.98, 25.09, 58.54, 24.13, 45.42, 29.99, 26.25, 34.46, 34.41, 38.74, 26.22, 25.1, 20.94, 28.89, 39.71, 23.67, 17.47, 39.53, 33.65, 28.97, 28.73, 37.59, 25.83, 36.49, 23.39, 45.33, 27.44, 33.2, 23.73, 31.48, 27.73, 23.3, 32.14, 23.34, 31.19, 23.24, 32.92, 31.75, 19.74, 39.99, 34.67, 22.14,

In [ ]:
dataset_resultado

,Transición 1,Nodo Intermedio,Transición 2,Nodo Final
0,55-59,BMI,16.60,No
1,55-59,BMI,20.34,No
2,55-59,BMI,26.58,No
3,55-59,BMI,24.21,No
4,55-59,BMI,23.71,No
...,...,...,...,...
23943,25-29,BMI,55.36,No
23944,25-29,BMI,51.84,No
23945,25-29,BMI,26.49,No
23946,25-29,BMI,59.90,No
